# Testing Aigents Sentiment Analysis Web API

In [1]:
import os, sys, math
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

import datetime as dt

if 'pygents.aigents_api' in sys.modules:
    del sys.modules['pygents.aigents_api']
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']

from pygents.aigents_api import AigentsSentiment, PygentsSentiment, TextMetrics
from pygents.util import vector_proximity


def language_metrics(lang,metrics_list):
    metrics = {}
    for m in metrics_list:
        metrics[m] = './data/dict/' + lang + '/' + m + '.txt'
    return metrics

        
p = PygentsSentiment('./data/dict/en/positive.txt',
                     './data/dict/en/negative.txt',debug=True)
a = TextMetrics(language_metrics('en',['catastrophizing','dichotomous-reasoning']),debug=True)
s = TextMetrics(language_metrics('en',['positive','negative']),debug=True)

print(a.gram_arity)
print(s.gram_arity)


4
6


In [2]:
a.metrics


{'catastrophizing': {('will', 'be', 'a', 'catastrophe'),
  ('will', 'be', 'a', 'disaster'),
  ('will', 'be', 'horrible'),
  ('will', 'be', 'impossible'),
  ('will', 'be', 'terrible'),
  ('will', 'end'),
  ('will', 'fail'),
  ('will', 'go', 'wrong'),
  ('will', 'never', 'end'),
  ('will', 'not', 'end'),
  ('will', 'not', 'happen')},
 'dichotomous-reasoning': {('all',),
  ('always',),
  ('black', 'or', 'white'),
  ('either', 'or'),
  ('ever',),
  ('every',),
  ('everybody',),
  ('everyone',),
  ('everything',),
  ('everywhere',),
  ('neither',),
  ('never',),
  ('no', 'one'),
  ('nobody',),
  ('nor',),
  ('not', 'a', 'single'),
  ('nothing',),
  ('nowhere',),
  ('only',),
  ('perfect',),
  ('the', 'best'),
  ('the', 'worst'),
  ('worthless',)}}

In [3]:
for m in s.metrics:
    ngrams = s.metrics[m]
    for n in ngrams:
        if len(n) > 4:
            print(n)


('revelation', 'of', 'saint', 'john', 'the', 'divine')
('feel', 'like', 'a', 'million', 'dollars')
('too', 'big', 'for', "one's", 'breeches')
('pull', 'a', 'fast', 'one', 'on')
('respiratory', 'distress', 'syndrome', 'of', 'the', 'newborn')


In [8]:
texts = ['good news','bad news','good bad news','good and pleasant news about bad things','empty spaces']
for t in texts:
    sentiment = p.get_sentiment(t)
    metrics = s.get_sentiment_words(t)
    assert sentiment[1] == (metrics['positive'] if 'positive' in metrics else 0) 
    assert sentiment[2] == (-metrics['negative'] if 'negative' in metrics else 0) 
    print(sentiment,metrics,t)


(0.85, 0.85, -0.0, 0.0, 2, 1) {'positive': 0.85} good news
(-0.85, 0.0, -0.85, 0.0, 2, 1) {'negative': 0.85} bad news
(0.0, 0.77, -0.77, 0.77, 3, 1) {'positive': 0.77, 'negative': 0.77} good bad news
(0.14, 0.74, -0.59, 0.66, 7, 1) {'positive': 0.74, 'negative': 0.59} good and pleasant news about bad things
(0.0, 0.0, -0.0, 0.0, 2, 1) {} empty spaces


In [23]:
texts = ['empty spaces',
         'i will fail',
         'i will fail without of glass of water',
         'i will fail without of glass of water and bottle of whiskey',
         'i will fail without of glass of water and bottle of whiskey every day',
         'never ever',
         'damn cold']
for t in texts:
    metrics = a.get_sentiment_words(t)
    print(metrics,t)


{} empty spaces
{'catastrophizing': 0.92} i will fail
{'catastrophizing': 0.71} i will fail without of glass of water
{'catastrophizing': 0.62} i will fail without of glass of water and bottle of whiskey
{'catastrophizing': 0.59, 'dichotomous-reasoning': 0.46} i will fail without of glass of water and bottle of whiskey every day
{'dichotomous-reasoning': 1.0} never ever
{} damn cold


In [25]:
labels = ['positive','negative','rude',
'catastrophizing','dichotomous-reasoning','disqualifying-positive','emotional-reasoning','fortune-telling',
'labeling','magnification','mental-filtering','mindreading','overgeneralizing','personalizing','should-statement']
l = TextMetrics(language_metrics('en',labels),debug=False)
for t in texts:
    metrics = l.get_sentiment_words(t)
    print(metrics,t)


{} empty spaces
{'catastrophizing': 0.92} i will fail
{'catastrophizing': 0.71} i will fail without of glass of water
{'catastrophizing': 0.62} i will fail without of glass of water and bottle of whiskey
{'catastrophizing': 0.59, 'dichotomous-reasoning': 0.46} i will fail without of glass of water and bottle of whiskey every day
{'negative': 0.85, 'dichotomous-reasoning': 1.0} never ever
{'negative': 1.0, 'rude': 0.85} damn cold
